
# Classification using Gradio

In [2]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of selenium to determine which version is compatible with other requirements. This

In [3]:
# Install required packages
!pip install gradio

In [4]:
# Step 1: Import the necessary libraries
import pandas as pd
from pycaret.classification import *
import gradio as gr

# Step 2: Load the Iris dataset
from pycaret.datasets import get_data
data = get_data('iris')

# Step 3: Set up the PyCaret classification experiment
exp = setup(data, target='species',  session_id=123)

# Step 4: Train the best model using compare_models()
best_model = compare_models()

# Step 5: Finalize the model (train on the full dataset)
final_model = finalize_model(best_model)

# Step 6: Save the model to disk
save_model(final_model, 'final_iris_model')

# Load the model back (for deployment, it's good practice)
model = load_model('final_iris_model')

# Step 7: Define the prediction function with debugging to inspect column names
def predict(sepal_length, sepal_width, petal_length, petal_width):
    try:
        # Prepare the input data as a DataFrame
        input_data = pd.DataFrame([[sepal_length, sepal_width, petal_length, petal_width]],
                                  columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])

        # Debugging: Print input data
        print(f"Input Data:\n{input_data}")

        # Use the loaded model to make predictions
        prediction = predict_model(model, data=input_data)

        # Debugging: Print prediction output to inspect the columns
        print(f"Prediction Output:\n{prediction}")

        # Check if 'Label' column exists, else return first column
        if 'Label' in prediction.columns:
            return prediction['Label'][0]
        else:
            # Return the first column name dynamically
            return prediction.iloc[:, -1][0]  # Adjust this to return the last column (likely the prediction)

    except Exception as e:
        # If there's an error, return the error message for debugging
        return f"Error: {e}"

# Step 8: Create the Gradio Interface with proper input ranges
sepal_length = gr.Slider(minimum=4.3, maximum=7.9, value=5.1, label='Sepal Length')
sepal_width = gr.Slider(minimum=2.0, maximum=4.4, value=3.5, label='Sepal Width')
petal_length = gr.Slider(minimum=1.0, maximum=6.9, value=1.4, label='Petal Length')
petal_width = gr.Slider(minimum=0.1, maximum=2.5, value=0.2, label='Petal Width')

# Create the Gradio interface with the trained model
gr.Interface(fn=predict,
             inputs=[sepal_length, sepal_width, petal_length, petal_width],
             outputs="text",
             title="Iris Flower Species Prediction").launch()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


,Description,Value
0,Session id,123
1,Target,species
2,Target type,Multiclass
3,Target mapping,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virginica: 2"
4,Original data shape,"(150, 5)"
5,Transformed data shape,"(150, 5)"
6,Transformed train set shape,"(105, 5)"
7,Transformed test set shape,"(45, 5)"
8,Numeric features,4
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9718,0.0000,0.9718,0.9780,0.9712,0.9573,0.9609,1.1490
knn,K Neighbors Classifier,0.9718,0.9830,0.9718,0.9780,0.9712,0.9573,0.9609,0.0510
qda,Quadratic Discriminant Analysis,0.9718,0.0000,0.9718,0.9780,0.9712,0.9573,0.9609,0.0390
lda,Linear Discriminant Analysis,0.9718,0.0000,0.9718,0.9780,0.9712,0.9573,0.9609,0.0340
lightgbm,Light Gradient Boosting Machine,0.9536,0.9935,0.9536,0.9634,0.9528,0.9298,0.9356,0.5840
nb,Naive Bayes,0.9445,0.9868,0.9445,0.9525,0.9438,0.9161,0.9207,0.0350
et,Extra Trees Classifier,0.9445,0.9935,0.9445,0.9586,0.9426,0.9161,0.9246,0.1710
catboost,CatBoost Classifier,0.9445,0.9922,0.9445,0.9586,0.9426,0.9161,0.9246,0.7880
gbc,Gradient Boosting Classifier,0.9355,0.0000,0.9355,0.9416,0.9325,0.9023,0.9083,0.2910
dt,Decision Tree Classifier,0.9264,0.9429,0.9264,0.9502,0.9201,0.8886,0.9040,0.0430


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://01812a926035ce8eeb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
